In [39]:
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

import config

In [40]:
import pandas as pd
from openai import OpenAI
import json
import random
from tqdm import tqdm

In [41]:
df_players = pd.read_csv('../../data/players_stats.csv')
df_players

,Unnamed: 0,index,league,season,team,player,nation__standard,pos__standard,age__standard,born__standard,...,Performance_Crs_misc,Performance_Int_misc,Performance_TklW_misc,Performance_PKwon_misc,Performance_PKcon_misc,Performance_OG_misc,Performance_Recov_misc,Aerial Duels_Won_misc,Aerial Duels_Lost_misc,Aerial Duels_Won%_misc
0,0,0,ENG-Premier League,2425,Arsenal,Ben White,ENG,DF,26.0,1997.0,...,17,16,14,0,0,0,37,18,9,66.7
1,1,1,ENG-Premier League,2425,Arsenal,Bukayo Saka,ENG,"FW,MF",22.0,2001.0,...,117,3,15,1,0,0,70,10,20,33.3
2,2,2,ENG-Premier League,2425,Arsenal,David Raya,ESP,GK,28.0,1995.0,...,0,2,0,0,1,0,41,13,0,100.0
3,3,3,ENG-Premier League,2425,Arsenal,Declan Rice,ENG,MF,25.0,1999.0,...,164,25,25,0,0,0,156,37,25,59.7
4,4,4,ENG-Premier League,2425,Arsenal,Ethan Nwaneri,ENG,"FW,MF",17.0,2007.0,...,54,2,7,0,0,0,34,4,8,33.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2849,2849,2849,ITA-Serie A,2425,Venezia,Nicholas Pierini,ITA,MF,25.0,1998.0,...,3,0,0,0,0,0,2,1,3,25.0
2850,2850,2850,ITA-Serie A,2425,Venezia,Nunzio Lella,ITA,MF,24.0,2000.0,...,0,0,0,0,0,0,0,0,0,NaN
2851,2851,2851,ITA-Serie A,2425,Venezia,Richie Sagrado,BEL,DF,20.0,2004.0,...,0,3,2,0,0,0,6,2,0,100.0
2852,2852,2852,ITA-Serie A,2425,Venezia,Ridgeciano Haps,SUR,DF,31.0,1993.0,...,26,7,16,0,1,0,54,14,18,43.8


In [42]:
exclude_columns = ['season', 'team', 'player', 'nation__standard', 'age__standard', 'born__standard', 'nation__shooting', 'pos__shooting', 'age__shooting', 'born__shooting', 'nation__passing', 'pos__passing', 'age__passing', 'born__passing', 'nation__defense', 'pos__defense', 'age__defense', 'born__defense', 'nation__possession', 'pos__possession', 'age__possession', 'born__possession', 'nation__misc', 'pos__misc', 'age__misc', 'born__misc']

In [43]:
df_players_filtered = df_players[df_players['Playing Time_90s_standard'] >= 8]
df_players_filtered = df_players_filtered.reset_index()
df_players_filtered.shape

(1735, 159)

In [44]:
client = OpenAI(api_key=config.Config.OPENAI_API_KEY)
player_summaries = {}

In [48]:
for _, row in tqdm(df_players_filtered.iterrows()):
    player_key = f"{row['player']} ({row['team']})"
    if player_key in player_summaries:
        print(f"✅ {player_key} → Summary exist")
        continue
    
    summary_prompt = f"""
        Tu es un expert en scouting football, analyste football spécialisé en scouting basé sur les données.
        
        Tu vas recevoir les statistiques détaillées d’un joueur professionnel.  
        À partir de ces données, rédige une **fiche de scouting complète, claire et neutre**, destinée à alimenter une base de recherche intelligente.

        ---

        ### 🎯 Objectif du texte :
        Identifier avec précision :
        - le **rôle du joueur** (poste et sous-rôle)
        - son **style de jeu** (comportement avec et sans ballon)
        - ses **principales qualités**
        - ses **axes d'amélioration**
        - son **profil-type** en fin de texte

        Le résumé doit permettre à un recruteur ou analyste d’avoir une **vision rapide mais fiable** du profil du joueur.

        ---
        
        ### ✍️ Structure attendue (fluide, sans titres ni bullets) :

        1. **Style de jeu et rôle** :  
        Décris le poste principal (DF, MF, FW…) et si possible, deduis des stats disponibles, le sous-rôle (ex : latéral offensif, milieu récupérateur, attaquant mobile).  
        Mentionne comment il évolue : projections, appels, conservation, pressing, largeur, implication défensive…  
        Base-toi uniquement sur les statistiques fournies.

        2. **Qualités principales** :  
        Regroupe par catégories :
        - **Physique** : volume de jeu, duels gagnés, présence, mobilité…
        - **Technique** : types de passes, créativité, conduite, finition, dribbles…
        - **Mental/tactique** : implication, pressing, discipline, concentration…

        3. **Axes d'amélioration** :  
        Exprime de manière neutre les domaines où il est moins à l’aise ou peut progresser.  
        Utilise des tournures comme “gagnerait à améliorer…”, “peut encore progresser sur…”, “montre quelques limites dans…”

        4. **Profil-type** :  
        Termine toujours par une ligne :  
        **Profil-type :** suivi d’une expression courte (3 à 6 mots) qui résume son profil.  
        (ex : “Milieu défensif intense”, “Défenseur axial sobre et solide”, “Ailier percutant et créatif”)

        ---

        ### ⚠️ Contraintes essentielles :

        - **Ne fais aucune déduction** à partir de données absentes (taille, puissance, nationalité, âge, etc.)
        - **N'invente jamais** une information
        - Ne donne aucun chiffre, pourcentage ou ratio
        - Texte fluide, sans bullet points, environ 6 à 8 phrases
        - Ne mentionne jamais le nom, le club, la ligue ou la nationalité du joueur

        ---
        
        IMPORTANT :
        - Ne déduis ni n’invente aucune donnée.
        - Si une information n’est pas présente dans les statistiques (comme la taille, la vitesse, la puissance, etc.), **n’en parle pas**.
        - Reste strictement fidèle aux données fournies.

        Voici les données du joueur :

        {row.drop(exclude_columns).to_string()}
    """
    
    response = client.chat.completions.create(
        model = config.Config.OPENAI_MODEL,
        messages=[
            {"role": "system", "content": "Tu es un expert en data scouting et en analyse footballistique. Capable de decrypter le style de jeu d'un joueur de part ses stats"},
            {"role": "user", "content": summary_prompt}
        ],
        temperature=0.2,
        max_tokens=500
    )
    
    player_summary = response.choices[0].message.content.strip()
    player_summaries[player_key] = player_summary
    print(f"✅ {player_key} → {player_summary}")
    
    if len(player_summaries) % 50 == 0:
        with open("../../data/player_summaries.json", "w", encoding="utf-8") as f:
            json.dump(player_summaries, f, ensure_ascii=False, indent=2)
            print('📸 Snapshot')
    
player_summaries

0it [00:00, ?it/s]

✅ Ben White (Arsenal) → Summary exist
✅ Bukayo Saka (Arsenal) → Summary exist
✅ David Raya (Arsenal) → Summary exist
✅ Declan Rice (Arsenal) → Summary exist
✅ Ethan Nwaneri (Arsenal) → Summary exist
✅ Gabriel Magalhães (Arsenal) → Summary exist
✅ Gabriel Martinelli (Arsenal) → Summary exist
✅ Jakub Kiwior (Arsenal) → Summary exist
✅ Jurriën Timber (Arsenal) → Summary exist
✅ Kai Havertz (Arsenal) → Summary exist
✅ Leandro Trossard (Arsenal) → Summary exist
✅ Martin Ødegaard (Arsenal) → Summary exist
✅ Mikel Merino (Arsenal) → Summary exist
✅ Myles Lewis-Skelly (Arsenal) → Summary exist
✅ Riccardo Calafiori (Arsenal) → Summary exist
✅ Thomas Partey (Arsenal) → Summary exist
✅ William Saliba (Arsenal) → Summary exist
✅ Amadou Onana (Aston Villa) → Summary exist
✅ Boubacar Kamara (Aston Villa) → Summary exist
✅ Diego Carlos (Aston Villa) → Summary exist
✅ Emiliano Martínez (Aston Villa) → Summary exist
✅ Ezri Konsa (Aston Villa) → Summary exist
✅ Ian Maatsen (Aston Villa) → Summary exist


1536it [00:07, 216.04it/s]

✅ Kenan Yıldız (Juventus) → Le joueur évolue principalement en tant qu'attaquant ou milieu de terrain, avec une tendance à participer activement à la construction du jeu et à l'attaque. Son style de jeu se caractérise par une implication offensive significative, illustrée par un nombre élevé de touches dans le dernier tiers et une capacité à créer des occasions de but. Il montre également une certaine polyvalence, capable de décrocher pour participer à la circulation du ballon tout en se projetant vers l'avant pour finaliser les actions.

Parmi ses qualités principales, on note un bon volume de jeu avec un nombre élevé de passes complètes et une précision notable, notamment dans les passes courtes. Sa capacité à dribbler et à prendre des risques dans les zones avancées est également un atout, avec un taux de réussite acceptable dans ses prises de balle. Sur le plan défensif, il participe activement aux récupérations et aux pressings, bien qu'il montre une certaine fragilité dans les du

1539it [00:26, 44.20it/s] 

...

1734it [23:21,  7.99s/it]

✅ Mikael Ellertsson (Venezia) → Le joueur évolue principalement en tant que défenseur, avec des capacités à jouer également au milieu de terrain, ce qui lui permet d'adopter un rôle polyvalent sur le terrain. Son style de jeu se caractérise par une forte implication défensive, avec une bonne capacité à récupérer le ballon et à effectuer des interventions dans les trois tiers du terrain. Il se projette également vers l'avant, contribuant à la construction du jeu et à la création d'opportunités, tout en maintenant une certaine largeur dans le jeu.

Parmi ses qualités principales, on note une bonne présence physique, illustrée par un volume de jeu élevé et une capacité à gagner des duels. Sur le plan technique, il se distingue par sa capacité à réaliser des passes précises, notamment dans les zones de construction, et par une certaine créativité dans ses actions offensives. Mentalement, il fait preuve d'une bonne discipline et d'une concentration constante, ce qui lui permet de maintenir 

1735it [23:29,  1.23it/s]

✅ Ridgeciano Haps (Venezia) → Le joueur évolue principalement en tant que défenseur, avec un sous-rôle qui semble être celui d'un défenseur central capable de contribuer à la construction du jeu. Son style de jeu se caractérise par une implication défensive solide, avec une bonne capacité à récupérer le ballon et à bloquer les passes adverses. Il montre également une certaine capacité à progresser avec le ballon, ce qui lui permet de participer à la phase offensive de son équipe, notamment par des passes précises et des projections vers l'avant.

Parmi ses principales qualités, on note un bon volume de jeu avec un nombre significatif de duels défensifs et une capacité à réaliser des interceptions. Sur le plan technique, il se distingue par une bonne précision dans ses passes, notamment à courte et moyenne distance, et une capacité à effectuer des passes clés. Mentalement, il fait preuve d'une bonne discipline et d'une concentration constante, ce qui lui permet de rester impliqué dans l

{'Ben White (Arsenal)': "Le joueur évolue principalement en tant que défenseur, avec un sous-rôle qui pourrait être celui d'un défenseur central solide, capable de participer à la construction du jeu tout en restant vigilant défensivement. Ses statistiques indiquent une implication active dans le jeu, avec un volume de passes élevé et une capacité à progresser avec le ballon, ce qui suggère qu'il prend des initiatives offensives tout en maintenant une bonne couverture défensive. Il semble également être impliqué dans le pressing, cherchant à récupérer le ballon rapidement.\n\nParmi ses principales qualités, on note une bonne capacité physique, illustrée par un nombre significatif de duels aériens remportés et une présence dans les duels au sol. Sur le plan technique, il affiche une bonne précision dans ses passes, notamment dans les courtes et moyennes distances, et il est capable de créer des occasions pour ses coéquipiers, comme en témoigne son nombre d'assists. Mentalement, il démon

In [49]:
len(player_summaries)

1735

In [50]:
with open("../../data/player_summaries.json", "w", encoding="utf-8") as f:
    json.dump(player_summaries, f, ensure_ascii=False, indent=2)